"A word search is a grid of letters with hidden words placed along rows, columns, and diagonals. A player of a word-search puzzle attempts to find the hidden words by carefully scanning through the grid. Finding places to put the words so that they all fit on the grid is a kind of constraint-satisfaction problem. The variables are the words, and the domains are the possible locations of those words."

In [1]:
from typing import NamedTuple, List, Dict, Optional
from random import choice
from string import ascii_uppercase
from csp import CSP, Constraint

Grid = List[List[str]] # type alias for grids

class GridLocation(NamedTuple):
    row: int
    column: int
        
# we fill the grid with letters fo the English alphabet
def generate_grid(rows: int, columns: int) -> Grid:
    # initialise grid with random letters
    return [[choice(ascii_uppercase) for c in range(columns)] for r in range(rows)]

# a function to display the grid
def display_grid(grid: Grid) -> None:
    for row in grid:
        print(''.join(row))

"The domain of a word is a list of lists of the possible locations of all of its letters (__List[List[GridLocation]]__). Words cannot just go anywhere, though. They must stay within a row, column, or diagonal that is within the bounds of the grid. In other words, they should not go off the end of the grid. The purpose of __generate_domain()__ is to build these lists for every word." aka, every word will have a domain of the entire grid.

In [2]:
def generate_domain(word: str, grid: Grid) -> List[List[GridLocation]]:
    domain: List[List[GridLocation]] = []
    height: int = len(grid) # rows
    width: int = len(grid[0]) #columns
    length: int = len(word)
    for row in range(height):
        for col in range(width):
        # a word can go along its column or row
        
            columns: range = range(col, col + length + 1)
            rows: range = range(row, row + length + 1)
            if col + length <= width:
                # Left to right
                domain.append([GridLocation(row, c) for c in columns]) 
                # diagonal towards bottom right
                if row + length <= height:
                    domain.append([GridLocation(r, col + (r - row)) for r in rows])
            if row + length <= height:
                # top to bottom
                domain.append([GridLocation(r, col) for r in rows])
                # diagonal towards bottom left
                if col - length >= 0:
                    domain.append([GridLocation(r, col - (r - row)) for r in rows])
    return domain

In our custom constraint class __WordSearchConstraint__ we will have the __satisfied__ method check whether any of the locations proposed for one word are the same as a location proposed for another. To a achieve this, we use __set__'s to remove duplicates.

In [3]:
class WordSearchConstraint(Constraint[str, List[GridLocation]]):
    # our variable is word and domain the grid as a list
    
    def __init__(self, words: List[str]) -> None:
        super().__init__(words)
        self.words: List[str] = words
            
    def satisfied(self, assignment: Dict[str, List[GridLocation]]) -> bool:
        # if there are any duplicated grid locations, then there is an overlap
        # we use 2 sets of locs for values because our grid is a list of lists
        all_locations = [locs for values in assignment.values() for locs in values]
        return len(set(all_locations)) == len(all_locations)

We will assign 5 words to a 9x9 grid:

In [8]:
grid: Grid = generate_grid(9,9)
display_grid(grid)

CNOSJERKO
ZGQZUUTGT
WRPHSMVAW
AWHEZATCP
WOHIDRRLZ
TEUHEPLMD
ZSUODSCXH
XBTSLUIGZ
BOKZATOMY


In [6]:
words: List[str] = ['MATTHEW', 'JOE', 'MARY', 'SARAH', 'SALLY']
locations: Dict[str, List[List[GridLocation]]] = {}

for word in words:
    locations[word] = generate_domain(word, grid)
    
# example
locations['JOE']

[[GridLocation(row=0, column=0),
  GridLocation(row=0, column=1),
  GridLocation(row=0, column=2),
  GridLocation(row=0, column=3)],
 [GridLocation(row=0, column=0),
  GridLocation(row=1, column=1),
  GridLocation(row=2, column=2),
  GridLocation(row=3, column=3)],
 [GridLocation(row=0, column=0),
  GridLocation(row=1, column=0),
  GridLocation(row=2, column=0),
  GridLocation(row=3, column=0)],
 [GridLocation(row=0, column=1),
  GridLocation(row=0, column=2),
  GridLocation(row=0, column=3),
  GridLocation(row=0, column=4)],
 [GridLocation(row=0, column=1),
  GridLocation(row=1, column=2),
  GridLocation(row=2, column=3),
  GridLocation(row=3, column=4)],
 [GridLocation(row=0, column=1),
  GridLocation(row=1, column=1),
  GridLocation(row=2, column=1),
  GridLocation(row=3, column=1)],
 [GridLocation(row=0, column=2),
  GridLocation(row=0, column=3),
  GridLocation(row=0, column=4),
  GridLocation(row=0, column=5)],
 [GridLocation(row=0, column=2),
  GridLocation(row=1, column=3),
  G

In [7]:
csp: CSP[str, List[GridLocation]] = CSP(words, locations)

csp.add_constraint(WordSearchConstraint(words))

In [9]:
solution: Optional[Dict[str, List[GridLocation]]] = csp.backtracking_search()
if solution is None:
    print('No solution found!')
else:
    for word, grid_locations in solution.items():
        # random reverse half the time
        if choice([True, False]):
            grid_locations.reverse()
        for index, letter in enumerate(word):
            (row, col) = (grid_locations[index].row, grid_locations[index].column)
            grid[row][col] = letter
    display_grid(grid)

MATTHEWKO
ZYRAMUTGE
WRPHSHYAO
AWHEZALCJ
WOHIDRLLZ
TEUHEAAMD
ZSUODSSXH
XBTSLUIGZ
BOKZATOMY
